In [1]:
import os, numpy as np, logging
from pyapphot.apphot import starPSF
os.chdir('test_data')

Ginga not installed, use other viewer, or no viewer


# Calculate PSF properties at given co-ordinates
PSF properties which are required for checking the quality of the images and also for photometry are:\
FWHM, peak,ellipticity, pa ,background-sky noise. You can calculate all the properties using the starPSF class and save the results to a pickle file which can be opened later.

In [7]:
psf = starPSF(image='al@imlist', coordfile='new*.coo', calculate=True, saveto='imstat.pkl') 

xc=263.5863	yc=1145.8722
xc=565.6814	yc=894.2403
xc=620.2962	yc=1742.2601
xc=1287.2993	yc=404.1012
xc=263.9450	yc=1145.9830
xc=566.0164	yc=894.1403
xc=620.6364	yc=1742.3525
xc=1287.6292	yc=403.7495
xc=264.0484	yc=1145.9681
xc=566.1555	yc=894.1985
xc=620.8098	yc=1742.3980
xc=1287.6820	yc=404.0553
xc=264.1859	yc=1146.2991
xc=566.3073	yc=894.4842
xc=620.8736	yc=1742.8035
xc=1287.8734	yc=404.2877
xc=263.9702	yc=1146.1225
xc=566.1323	yc=894.3431
xc=620.7268	yc=1742.5998
xc=1287.6999	yc=404.1697


# Load PSF stat and analyze image quality

In [8]:
psf =  starPSF(image='al@imlist', coordfile='new*.coo', loadfrom='imstat.pkl')
psfstats = psf.get_stat('all', statkey=['ellip','peak','fwhm'], return_dict=True) # check some properties
# or just
# starPSF.get_stat(None,statkey=['ellip','peak','fwhm']) # check some properties
print(psfstats)

{'ellip': array([[0.09, 0.09, 0.11, 0.12],
       [0.03, 0.02, 0.04, 0.03],
       [0.13, 0.12, 0.14, 0.13],
       [0.16, 0.16, 0.18, 0.19],
       [0.15, 0.14, 0.16, 0.16]]), 'peak': array([[246794.67950891, 189438.12823083, 328185.93829712,
        465318.46940319],
       [245584.0259947 , 182785.77592538, 311608.67160259,
        430730.82211452],
       [234719.5667304 , 171933.70131579, 291833.12214311,
        403065.11028365],
       [260910.64904785, 192460.90881026, 322805.25212256,
        460695.74529167],
       [231548.3311077 , 173136.50063555, 288777.09962197,
        414802.57904973]]), 'fwhm': array([[6.312, 6.168, 6.08 , 6.   ],
       [6.268, 6.648, 6.624, 6.72 ],
       [6.336, 6.86 , 6.956, 6.924],
       [6.716, 7.056, 7.112, 6.824],
       [6.304, 6.676, 6.444, 6.716]]), 'image': ['alh2acl310100.fits', 'alh2acl310101.fits', 'alh2acl310102.fits', 'alh2acl310103.fits', 'alh2acl310104.fits'], 'coordfile': ['new_h2acl310100.coo', 'new_h2acl310101.coo', 'new_h2acl31

You can impose criteria and select the objects and frames accordingly. You can set a trade-off between removing bad objects and bad frames by the argument 'Nfiledelmax'. This tells you how many frames you are ready to remove from your calculation at most. Accordingly the objects not matching the criteria will be removed.This analysis will give you a list of frames that satisfy the given criteria for each object and also tells you how many frames and how many objects you need to remove from your calculation.

In [9]:
criteria = dict(ellip=(0,0.13),peak=(80000,500000))
imselected, remove = psf.select_imageNobj(criteria, objsynced=True, Nfiledelmax=2)
print("\nFollowing files are selected per object\n")
for i,im in enumerate(imselected):
    print(f"Object {i+1}: {', '.join(im)}")
remove

object 1: conditions not met by 2 image(s).
object 2: conditions not met by 2 image(s).
object 3: conditions not met by 3 image(s).
object 4: conditions not met by 2 image(s).

Following files are selected per object

Object 1: alh2acl310100.fits, alh2acl310101.fits, alh2acl310102.fits
Object 2: alh2acl310100.fits, alh2acl310101.fits, alh2acl310102.fits
Object 3: alh2acl310100.fits, alh2acl310101.fits
Object 4: alh2acl310100.fits, alh2acl310101.fits, alh2acl310102.fits


{'delete_images': ['alh2acl310103.fits', 'alh2acl310104.fits'],
 'remove_obj': [2]}

In [10]:
psf.adjust_coordfile(remove['remove_obj'])